In [1]:
import pandas as pd
import numpy as np

In [2]:
listings = pd.read_csv('tpf_listings.csv')
listings.shape

(20085, 75)

In [3]:
## Hice los análisis individuales x variable y está en el excel diccionario.xlsx - Asi que acá la preparación
## Genero un nuevo data frame 'listing_modelo' 
## ['host_since] lo convierto a años de antiguedad del host respecto al 2022-12-31
listings_modelo = pd.DataFrame()
listings_modelo ['antiguedad_host'] =\
    (pd.to_datetime('2022-12-31') - pd.to_datetime(listings['host_since'])).dt.days/365

In [4]:
## ['host_response_time'] a numérico
response_time_to_numeric = {'within an hour': 1, 'within a few hours': 2, 'within a day': 3, 'a few days or more': 4}
listings_modelo['host_response_time'] = listings['host_response_time'].map(response_time_to_numeric)

In [5]:
## ['host_response_rate'] quito '%' y convierto a float
## ['host_acceptance_rate'] quito '%' y convierto a float
listings_modelo['host_response_rate'] = listings['host_response_rate'].str.replace('%','').astype(float)
listings_modelo['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%','').astype(float)

In [6]:
## ['host_is_superhost'] convierto a numérico
listings_modelo['host_is_superhost'] = listings['host_is_superhost'].replace({'t': 1, 'f':0})

In [7]:
## ['host_listings_count'] y ['host_total_listings_count] los paso sin conversión - son enteros
listings_modelo['host_listings_count'] = listings['host_listings_count']
listings_modelo['host_total_listings_count'] = listings['host_total_listings_count']

In [8]:
## ['host_has_profile_pic'] y ['host_identity_verified'] convierto a numérico
listings_modelo['host_has_profile_pic'] = listings['host_has_profile_pic'].replace({'t': 1, 'f':0})
listings_modelo['host_identity_verified'] = listings['host_identity_verified'].replace({'t': 1, 'f':0})

In [9]:
## dejo los 20 principales barrios y al resto los etiqueto como otro
neighbourhood_cleansed_20_princ = listings['neighbourhood_cleansed'].value_counts().nlargest(20).index
listings_modelo['neighbourhood_cleansed'] = listings['neighbourhood_cleansed'].apply(lambda x:\
                                                    x if x in neighbourhood_cleansed_20_princ else 'otro')

In [10]:
##['property_type] dejo las principales 8 categorías que suman el 93% y al resto la etiqueto como 'otro'
property_type_8_princ = listings['property_type'].value_counts().nlargest(8).index
listings_modelo['property_type'] = listings['property_type'].apply(lambda x: x if x in property_type_8_princ else 'otro')

In [11]:
## ['accommodates'] la paso sin convertir
listings_modelo['accommodates'] = listings['accommodates']

In [12]:
## analizo los 23 amenities que más se nombran que suman un poco menos del 50% de los registros
import ast
import itertools
listings_modelo['amenities_lista'] = listings['amenities'].apply(ast.literal_eval)
lista_amenities = list(itertools.chain(*listings_modelo['amenities_lista'].tolist()))
serie_amenities = pd.Series(lista_amenities).value_counts()[:23]
serie_amenities

Kitchen                       19164
Long term stays allowed       18606
Wifi                          18134
Essentials                    16902
Hangers                       15447
Dishes and silverware         15148
Hot water                     15080
Cooking basics                14490
Refrigerator                  13387
Hair dryer                    12577
Heating                       12427
Bed linens                    12341
Elevator                      12078
Air conditioning              11950
Iron                          11721
Microwave                     11236
Extra pillows and blankets     9070
Oven                           8734
Shampoo                        8628
Dedicated workspace            8613
TV                             8577
Bidet                          8530
Room-darkening shades          8520
dtype: int64

In [13]:
## armo las 23 columnas como dummies pero a mano
for i in range(0,23):
    listings_modelo[serie_amenities.index[i]] =\
        listings_modelo['amenities_lista'].apply(lambda x: 1 if serie_amenities.index[i] in x else 0)

In [14]:
## los paso como están xq están como enteros o float
listings_modelo['minimum_nights'] = listings['minimum_nights']
listings_modelo['number_of_reviews'] = listings['number_of_reviews']
listings_modelo['number_of_reviews_ltm'] = listings['number_of_reviews_ltm']
listings_modelo['number_of_reviews_l30d'] = listings['number_of_reviews_l30d']
listings_modelo['review_scores_rating'] = listings['review_scores_rating']
listings_modelo['review_scores_accuracy'] = listings['review_scores_accuracy']
listings_modelo['review_scores_cleanliness'] = listings['review_scores_cleanliness']
listings_modelo['review_scores_checkin'] = listings['review_scores_checkin']
listings_modelo['review_scores_communication'] = listings['review_scores_communication']
listings_modelo['review_scores_location'] = listings['review_scores_location']

In [15]:
## paso como ceros y uno a la posibilidad de reservar en el instante
listings_modelo['instant_bookable'] = listings['instant_bookable'].replace({'t': 1, 'f':0})

In [16]:
## estas serían las 3 variables targets - en el original como días libres y acá como ocupación para los prox 30, 60 y 90 días
listings_modelo['ocupacion_30'] = 1-listings['availability_30']/30
listings_modelo['ocupacion_60'] = 1-listings['availability_60']/60
listings_modelo['ocupacion_90'] = 1-listings['availability_90']/90

In [17]:
listings_modelo.shape

(20085, 50)

In [18]:
listings_modelo = listings_modelo.drop('amenities_lista', axis=1)

In [19]:
listings_modelo = listings_modelo.dropna().reset_index(drop=True)

In [20]:
listings_modelo.shape

(14637, 49)

In [21]:
listings_modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14637 entries, 0 to 14636
Data columns (total 49 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   antiguedad_host              14637 non-null  float64
 1   host_response_time           14637 non-null  float64
 2   host_response_rate           14637 non-null  float64
 3   host_acceptance_rate         14637 non-null  float64
 4   host_is_superhost            14637 non-null  int64  
 5   host_listings_count          14637 non-null  int64  
 6   host_total_listings_count    14637 non-null  int64  
 7   host_has_profile_pic         14637 non-null  int64  
 8   host_identity_verified       14637 non-null  int64  
 9   neighbourhood_cleansed       14637 non-null  object 
 10  property_type                14637 non-null  object 
 11  accommodates                 14637 non-null  int64  
 12  Kitchen                      14637 non-null  int64  
 13  Long term stays 

In [22]:
## hago dummies de barrios y tipo de propiedad
from sklearn.preprocessing import OneHotEncoder
categoricals = ['neighbourhood_cleansed', 'property_type']
enc = OneHotEncoder(drop='first')
X = listings_modelo[categoricals]
enc.fit(X)
enc.categories_

c:\Users\VivoBook\anaconda3\envs\DH\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[array(['Almagro', 'Balvanera', 'Barracas', 'Belgrano', 'Caballito',
        'Chacarita', 'Colegiales', 'Constitucion', 'Monserrat', 'Nuñez',
        'Palermo', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra',
        'San Cristobal', 'San Nicolas', 'San Telmo', 'Villa Crespo',
        'Villa Urquiza', 'otro'], dtype=object),
 array(['Entire condo', 'Entire home', 'Entire loft', 'Entire rental unit',
        'Entire serviced apartment', 'Entire vacation home',
        'Private room in home', 'Private room in rental unit', 'otro'],
       dtype=object)]

In [23]:
dummies = enc.transform(X).toarray()
dummies_df = pd.DataFrame(dummies)
col_names = [categoricals[i] + '_' + enc.categories_[i] for i in range(len(categoricals))]
col_names_drop_first = [sublist[i] for sublist in col_names for i in range(len(sublist)) if i !=0]
dummies_df.columns = col_names_drop_first
dummies_df

,neighbourhood_cleansed_Balvanera,neighbourhood_cleansed_Barracas,neighbourhood_cleansed_Belgrano,neighbourhood_cleansed_Caballito,neighbourhood_cleansed_Chacarita,neighbourhood_cleansed_Colegiales,neighbourhood_cleansed_Constitucion,neighbourhood_cleansed_Monserrat,neighbourhood_cleansed_Nuñez,neighbourhood_cleansed_Palermo,...,neighbourhood_cleansed_Villa Urquiza,neighbourhood_cleansed_otro,property_type_Entire home,property_type_Entire loft,property_type_Entire rental unit,property_type_Entire serviced apartment,property_type_Entire vacation home,property_type_Private room in home,property_type_Private room in rental unit,property_type_otro
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
14633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
14635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [24]:
listings_modelo_resto = listings_modelo.drop(categoricals, axis=1)

In [25]:
listings_modelo = pd.concat([listings_modelo_resto, dummies_df], axis=1)

In [26]:
listings_modelo.to_csv('tpf_airbnb_2022.csv')

In [27]:
listings_modelo.shape

(14637, 75)

In [28]:
## dejo explícitas las variables para trabajar los modelos

variables_dummies = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'Kitchen', 'Long term stays allowed',\
                    'Wifi', 'Essentials', 'Hangers', 'Dishes and silverware', 'Hot water', 'Cooking basics', 'Refrigerator',\
                    'Hair dryer', 'Heating', 'Bed linens', 'Elevator', 'Air conditioning', 'Iron', 'Microwave',\
                    'Extra pillows and blankets', 'Oven', 'Shampoo', 'Dedicated workspace', 'TV', 'Bidet', \
                    'Room-darkening shades','instant_bookable', 'neighbourhood_cleansed_Balvanera',\
                    'neighbourhood_cleansed_Barracas', 'neighbourhood_cleansed_Belgrano', 'neighbourhood_cleansed_Caballito',\
                    'neighbourhood_cleansed_Chacarita', 'neighbourhood_cleansed_Colegiales', 'neighbourhood_cleansed_Constitucion',\
                    'neighbourhood_cleansed_Monserrat', 'neighbourhood_cleansed_Nuñez', 'neighbourhood_cleansed_Palermo',\
                    'neighbourhood_cleansed_Palermo', 'neighbourhood_cleansed_Recoleta', 'neighbourhood_cleansed_Retiro',\
                    'neighbourhood_cleansed_Saavedra', 'neighbourhood_cleansed_San Cristobal', 'neighbourhood_cleansed_San Nicolas',\
                    'neighbourhood_cleansed_San Telmo', 'neighbourhood_cleansed_Villa Crespo', 'neighbourhood_cleansed_Villa Urquiza',\
                    'neighbourhood_cleansed_otro', 'property_type_Entire home', 'property_type_Entire loft',\
                    'property_type_Entire rental unit', 'property_type_Entire serviced apartment',\
                    'property_type_Entire vacation home', 'property_type_Private room in home',\
                    'property_type_Private room in rental unit', 'property_type_otro']


variables_numericas_normalizar = ['antiguedad_host', 'host_response_time', 'host_response_rate', 'host_acceptance_rate',\
                                  'host_listings_count', 'host_total_listings_count', 'accommodates',\
                                    'minimum_nights', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',\
                                    'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness'\
                                    'review_scores_checkin', 'review_scores_communication', 'review_scores_location']

variables_target = ['ocupacion_30', 'ocupacion_60', 'ocupacion_60']